# TensorFlow 一般流程及概念

## 基本使用

使用 TensorFlow, 必须明白 TensorFlow:
* 使用图 (graph) 来表示计算任务.
* 在被称之为 会话 (Session) 的上下文 (context) 中执行图.
* 使用 tensor 表示数据.
* 通过 变量 (Variable) 维护状态.
* 使用 feed 和 fetch 可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据.

### Tensor
Tensor的意思是张量，不过按我的理解，其实就是指矩阵。也可以理解为tensorflow中矩阵的表示形式。
###  Variable
变量的意思。一般用来表示图中的各计算参数，包括矩阵，向量等。例如：y=Relu(Wx+b)这里W和b是要用来训练的参数，那么此时这两个值就可以用Variable来表示。如：W = tf.Variable(tf.zeros(shape=[1,2]))
### placeholder
占位符，用于表示输入输出数据的格式。告诉系统：这里有一个值/向量/矩阵，现在我没法给你具体数值，不过我正式运行的时候会补上的！例如上式中的x和y。因为没有具体数值，所以只要指定尺寸即可

```
x = tf.placeholder(tf.float32,[1, 5],name='input')
y = tf.placeholder(tf.float32,[None, 5],name='input')
```

上面有两种形式，第一种x，表示输入是一个[1,5]的横向量。       
第二种形式，表示输入是一个[?,5]的矩阵。那么什么情况下会这么用呢?就是需要输入一批[1,5]的数据的时候。比如我有一批共10个数据，那我可以表示成[10,5]的矩阵。如果是一批5个，那就是[5,5]的矩阵。tensorflow会自动进行批处理
###  Session
session，也就是会话。我的理解是，session是抽象模型的实现者。为什么之前的代码多处要用到session？因为模型是抽象的嘛，只有实现了模型以后，才能够得到具体的值。同样，具体的参数训练，预测，甚至变量的实际值查询，都要用到session.

## 模型构建 (graph)
这里使用官方tutorial中的mnist数据集的分类代码，公式可以写作   
`
z=Wx+b
a=softmax(z)
`         
模型的代码描述为    
1. 建立抽象模型        
`
x = tf.placeholder(tf.float32, [None, 784]) # 输入占位符
y = tf.placeholder(tf.float32, [None, 10])  # 输出占位符（预期输出）
W = tf.Variable(tf.zeros([784, 10]))        
b = tf.Variable(tf.zeros([10]))
a = tf.nn.softmax(tf.matmul(x, W) + b)      # a表示模型的实际输出
`
2. 定义损失函数和训练方法    
`
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(a), reduction_indices=[1])) # 损失函数为交叉熵
optimizer = tf.train.GradientDescentOptimizer(0.5) # 梯度下降法，学习速率为0.5
train = optimizer.minimize(cross_entropy)  # 训练目标：最小化损失函数
`    
模型中的所有元素(图结构，损失函数，下降方法和训练目标)都已经包括在train里面。我们可以把train叫做训练模型。那么我们还需要测试模型     
`
correct_prediction = tf.equal(tf.argmax(a, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
`       
上述两行代码，tf.argmax表示找到最大值的位置(也就是预测的分类和实际的分类)，然后看看他们是否一致，是就返回true,不是就返回false,这样得到一个boolean数组。tf.cast将boolean数组转成float数组，最后求平均值，得到分类的准确率

3. 实际训练

有了训练模型和测试模型以后，我们就可以开始进行实际的训练了             
`
sess = tf.InteractiveSession()      # 建立交互式会话
tf.initialize_all_variables().run() # 所有变量初始化
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)    # 获得一批100个数据
    train.run({x: batch_xs, y: batch_ys})   # 给训练模型提供输入和输出
print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))
`     
可以看到，在模型搭建完以后，我们只要为模型提供输入和输出，模型就能够自己进行训练和测试了。中间的求导，求梯度，反向传播等等繁杂的事情，tensorflow都会帮你自动完成。

## 交互式使用
Python 使用一个会话 Session 来启动图, 并调用 Session.run() 方法执行操作.

为了便于使用诸如 IPython 之类的 Python 交互环境, 可以使用 InteractiveSession 代替 Session 类, 使用 Tensor.eval() 和 Operation.run() 方法代替 Session.run(). 这样可以避免使用一个变量来持有会话.

* 进入一个交互式 TensorFlow 会话.     
`
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])
`

* 使用初始化器 initializer op 的 run() 方法初始化 'x'    
`x.initializer.run()`      

* 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果     
`
sub = tf.sub(x, a)
print sub.eval()
[-2.,-1.]
`


## 保存与读取模型

在使用tf来训练模型的时候，难免会出现中断的情况。这时候自然就希望能够将辛辛苦苦得到的中间参数保留下来，不然下次又要重新开始。好在tf官方提供了保存和读取模型的方法。

* 保存模型

```
......各种构建模型graph的操作(矩阵相乘，sigmoid等等....)

saver = tf.train.Saver() # 生成saver

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # 先对模型初始化

    # 然后将数据丢入模型进行训练blablabla

    # 训练完以后，使用saver.save 来保存
    saver.save(sess, "save_path/file_name") #file_name如果不存在的话，会自动创建

```     

* 载入模型

```
saver = tf.train.Saver()

with tf.Session() as sess:
    #参数可以进行初始化，也可不进行初始化。即使初始化了，初始化的值也会被restore的值给覆盖
    sess.run(tf.global_variables_initializer())     
    saver.restore(sess, "save_path/file_name") #会将已经保存的变量值resotre到 变量中。

```    
**简单的说，就是通过saver.save来保存模型，通过saver.restore来加载模型**